In [ ]:
import pandas as pd
import numpy as np
import json
from Trend import read_pos_raw, pos_preprocess, convert_weekly_to_weekly_long
from rank import run_rank_batch_simple

In [ ]:
# POS path
POS_PATH = "Z:/CSS/POS Data/CVS.xlsx"
HEADER_ROW = 1  

brand = "CVS"

# 1) Raw POS loading
pos_raw = read_pos_raw(POS_PATH, header_row=HEADER_ROW)

# 2) pre processing part
pos_sel = pos_preprocess(pos_raw, brand=brand)

# 3) calculate weekly long format
weekly = convert_weekly_to_weekly_long(pos_sel, brand=brand)




### Trend Flag

In [ ]:
def attach_3w_point_trend_weekly_explainable(
    weekly: pd.DataFrame,
    pct_thr: float = 0.10,
    instock_min: float | None = None,
    min_valid_points: int = 3
) -> pd.DataFrame:

    df = weekly.copy()
    df = df.sort_values(["Material", "Year", "WeekNum"]).reset_index(drop=True)

    # Instock filter
    if instock_min is not None and "Instock" in df.columns:
        df["USW_for_trend"] = np.where(df["Instock"] >= instock_min, df["UPM_week"], np.nan)
    else:
        df["USW_for_trend"] = df["UPM_week"]

    
    df["base_year"] = df["Year"] if "Year" in df.columns else pd.NA
    df["base_weeknum"] = df["WeekNum"] if "WeekNum" in df.columns else pd.NA
    df["base_yearmonth"] = df["YearMonth"] if "YearMonth" in df.columns else pd.NaT

    # recent 3 weeks values
    df["v0_t_minus_2"] = np.nan
    df["v1_t_minus_1"] = np.nan
    df["v2_t"]         = np.nan

    # week details
    df["v0_week"] = pd.NA
    df["v1_week"] = pd.NA
    df["v2_week"] = pd.NA
    df["week_range_3w"] = pd.NA

    # change pct
    df["trend_pct_3w"] = np.nan

    # conditions
    df["cond_3w_up"] = False
    df["cond_3w_down"] = False
    df["cond_thr_up"] = False
    df["cond_thr_down"] = False

    # final labels
    df["trend_label_3w_point"] = pd.NA

    # calculate per material
    for mat, g_idx in df.groupby("Material").groups.items():
        idx_list = list(g_idx)  

        for j in range(len(idx_list)):
            base_idx = idx_list[j]

            # need min 3 points
            if j < 2:
                df.at[base_idx, "trend_label_3w_point"] = "NotEnoughData"
                continue

            win_indices = idx_list[j-2:j+1]  # t-2, t-1, t
            win = df.loc[win_indices].copy()

            valid_cnt = win["USW_for_trend"].notna().sum()
            if valid_cnt < min_valid_points:
                df.at[base_idx, "trend_label_3w_point"] = "NotEnoughData"
                continue

            v0 = win["USW_for_trend"].iloc[0]
            v1 = win["USW_for_trend"].iloc[1]
            v2 = win["USW_for_trend"].iloc[2]

            # weekNum
            w0 = win["WeekNum"].iloc[0]
            w1 = win["WeekNum"].iloc[1]
            w2 = win["WeekNum"].iloc[2]

            df.at[base_idx, "v0_t_minus_2"] = v0
            df.at[base_idx, "v1_t_minus_1"] = v1
            df.at[base_idx, "v2_t"]         = v2

            df.at[base_idx, "v0_week"] = f"W{int(w0)}" if pd.notna(w0) else pd.NA
            df.at[base_idx, "v1_week"] = f"W{int(w1)}" if pd.notna(w1) else pd.NA
            df.at[base_idx, "v2_week"] = f"W{int(w2)}" if pd.notna(w2) else pd.NA
            df.at[base_idx, "week_range_3w"] = f"W{int(w0)}~W{int(w2)}" if pd.notna(w0) and pd.notna(w2) else pd.NA

            # change pct (t vs t-2)
            pct = (v2 / v0 - 1) if pd.notna(v0) and v0 != 0 else np.nan
            df.at[base_idx, "trend_pct_3w"] = pct

            # condition
            cond_up = (v2 > v1 > v0)
            cond_dn = (v2 < v1 < v0)
            cond_thr_up = (pd.notna(pct) and pct >= pct_thr)
            cond_thr_dn = (pd.notna(pct) and pct <= -pct_thr)

            df.at[base_idx, "cond_3w_up"] = bool(cond_up)
            df.at[base_idx, "cond_3w_down"] = bool(cond_dn)
            df.at[base_idx, "cond_thr_up"] = bool(cond_thr_up)
            df.at[base_idx, "cond_thr_down"] = bool(cond_thr_dn)

            # labeling
            label = "Fluctuation"
            if cond_up and cond_thr_up:
                label = "Up"
            elif cond_dn and cond_thr_dn:
                label = "Down"

            df.at[base_idx, "trend_label_3w_point"] = label

    return df


weekly_labeled3 = attach_3w_point_trend_weekly_explainable(
    weekly,
    pct_thr=0.20,     # pct threshold for significant change
    instock_min=None  
)

weekly_trend = weekly_labeled3[['Material', 'Year', 'Month', 'WeekNum', 'UPM_week', 'v0_t_minus_2', 'v1_t_minus_1','v2_t', 'trend_label_3w_point']]
weekly_trend.rename(columns = {'UPM_week':'USW', 'v0_t_minus_2':'USW_2W', 'v1_t_minus_1':'USW_1W', 'v0_t':'USW', 'trend_label_3w_point':'Trend'}, inplace=True)


rca_trend = weekly_trend.copy()

### Rank

In [ ]:
df_rank = run_rank_batch_simple()

col = df_rank.columns[2]  
date_str = col.split('_')[1]

dt = pd.to_datetime(date_str)
df_rank['Year'] = dt.year
df_rank['Month'] = dt.month
df_rank['WeekNum'] = dt.isocalendar().week

df_rank_cvs = df_rank[df_rank['Customer']=='CVS']


### Instock

In [ ]:
weekly_instock = weekly[['Material', 'Year', 'Month', 'WeekNum', 'Instock']].sort_values(['Material', 'Year', 'Month', 'WeekNum'])

weekly_instock['Instock_lastweek'] = weekly_instock.groupby('Material')['Instock'].shift(1)


weekly_instock = weekly_instock.sort_values(["Material","Year","WeekNum"]).reset_index(drop=True)

weekly_instock["Instock_delta"] = weekly_instock["Instock"] - weekly_instock["Instock_lastweek"]


def attach_instock_var_flags_simple(
    weekly_instock: pd.DataFrame,
    vol_window: int = 8,          
    baseline_window: int = 20,   
    vol_z_thr: float = 1.5,        
    down_pct_thr: float = 0.10     
) -> pd.DataFrame:

    df = weekly_instock.copy()
    df = df.sort_values(["Material","Year","WeekNum"]).reset_index(drop=True)

    # delta
    df["Instock_delta"] = df["Instock"] - df["Instock_lastweek"]


    df["instock_vol_std"] = np.nan
    df["instock_vol_base_mean"] = np.nan
    df["instock_vol_base_std"] = np.nan
    df["instock_vol_z"] = np.nan

    df["Instock_t_minus_2"] = np.nan
    df["Instock_t_minus_1"] = np.nan
    df["Instock_t"] = np.nan
    df["instock_pct_3w"] = np.nan
    df["instock_3w_trend"] = pd.NA

    df["cond_vol_spike"] = False
    df["cond_3w_down"] = False
    df["cond_down_thr"] = False

    df["instock_flag_var"] = pd.NA

    for mat, g in df.groupby("Material"):
        idx = g.index.tolist()

        # Instock delta's rolling std = volatility
        delta = pd.to_numeric(df.loc[idx, "Instock_delta"], errors="coerce")
        vol = delta.rolling(vol_window, min_periods=max(3, vol_window//2)).std()

        # baseline
        base_mean = vol.rolling(baseline_window, min_periods=max(5, baseline_window//2)).mean()
        base_std  = vol.rolling(baseline_window, min_periods=max(5, baseline_window//2)).std()

        vol_z = (vol - base_mean) / base_std.replace({0: np.nan})

        df.loc[idx, "instock_vol_std"] = vol.values
        df.loc[idx, "instock_vol_base_mean"] = base_mean.values
        df.loc[idx, "instock_vol_base_std"]  = base_std.values
        df.loc[idx, "instock_vol_z"] = vol_z.values

        for j in range(len(idx)):
            base_idx = idx[j]

            # variance spike?
            vz = df.at[base_idx, "instock_vol_z"]
            cond_vol = pd.notna(vz) and (vz >= vol_z_thr)
            df.at[base_idx, "cond_vol_spike"] = bool(cond_vol)

            # 3 weeks down + down pct
            if j < 2:
                df.at[base_idx, "instock_3w_trend"] = "NotEnoughData"
                df.at[base_idx, "instock_flag_var"] = "NotEnoughData"
                continue

            v0 = df.at[idx[j-2], "Instock"]
            v1 = df.at[idx[j-1], "Instock"]
            v2 = df.at[idx[j],   "Instock"]

            df.at[base_idx, "Instock_t_minus_2"] = v0
            df.at[base_idx, "Instock_t_minus_1"] = v1
            df.at[base_idx, "Instock_t"] = v2

            if pd.isna(v0) or pd.isna(v1) or pd.isna(v2) or v0 == 0:
                df.at[base_idx, "instock_3w_trend"] = "NotEnoughData"
                df.at[base_idx, "instock_flag_var"] = "NotEnoughData"
                continue

            pct = (v2 / v0) - 1
            df.at[base_idx, "instock_pct_3w"] = pct

            cond_down = (v2 < v1 < v0)
            cond_down_thr = (pct <= -down_pct_thr)

            df.at[base_idx, "cond_3w_down"] = bool(cond_down)
            df.at[base_idx, "cond_down_thr"] = bool(cond_down_thr)

            if cond_down:
                df.at[base_idx, "instock_3w_trend"] = "Down"
            elif (v2 > v1 > v0):
                df.at[base_idx, "instock_3w_trend"] = "Up"
            else:
                df.at[base_idx, "instock_3w_trend"] = "Fluctuation"

            # final flag
            if cond_vol and cond_down and cond_down_thr:
                flag = "Volatile + Meaningful Down"
            elif cond_vol and cond_down:
                flag = "Volatile + Down"
            elif cond_vol:
                flag = "Volatile"
            elif cond_down and cond_down_thr:
                flag = "Stable but Meaningful Down"
            else:
                flag = "Stable/Noise"

            df.at[base_idx, "instock_flag_var"] = flag

    return df


weekly_instock_flagged = attach_instock_var_flags_simple(
    weekly_instock,
    vol_window=8,
    baseline_window=20,
    vol_z_thr=1.5,
    down_pct_thr=0.10
)
rca_instock = weekly_instock_flagged[[
    "Material","Year","Month","WeekNum",
    "Instock","Instock_lastweek","Instock_delta",
    "instock_vol_std",
    "instock_pct_3w","instock_3w_trend",
    "cond_vol_spike","cond_3w_down","cond_down_thr",
    "instock_flag_var"
]].copy()


### Final Merged Table

In [ ]:
df_merged = (
    df_rank_cvs
    .merge(rca_trend, on=["Material", "Year", "WeekNum"], how="left")
    .merge(rca_instock, on=["Material", "Year", "WeekNum"], how="left")
)
# rank/RankChange column collect
rank_cols = [c for c in df_rank_cvs.columns if c.startswith("Rank_")]
rank_change_cols = [c for c in df_rank_cvs.columns if c.startswith("RankChange_")]

def row_to_payload(r):
    return {
        "id": {
            "Material": r["Material"],
            "Customer": r["Customer"],
            "Year": int(r["Year"]),
            "WeekNum": int(r["WeekNum"]),
            "Month": int(r["Month"]) if pd.notna(r.get("Month")) else None,
        },
        "rank": {c: r.get(c) for c in (rank_cols + rank_change_cols)},
        "usw": {
            "USW": r.get("USW"),
            "USW_1W": r.get("USW_1W"),
            "USW_2W": r.get("USW_2W"),
            "Trend": r.get("Trend"),
        },
        "instock": {
            "Instock": r.get("Instock"),
            "Instock_lastweek": r.get("Instock_lastweek"),
            "Instock_delta": r.get("Instock_delta"),
            "instock_vol_std": r.get("instock_vol_std"),
            "instock_pct_3w": r.get("instock_pct_3w"),
            "instock_3w_trend": r.get("instock_3w_trend"),
            "cond_vol_spike": bool(r.get("cond_vol_spike")),
            "cond_3w_down": bool(r.get("cond_3w_down")),
            "cond_down_thr": bool(r.get("cond_down_thr")),
            "instock_flag_var": r.get("instock_flag_var"),
        },
    }
payloads = [row_to_payload(r) for _, r in df_merged.iterrows()]

# Save as JSONL 
with open("llm_input.jsonl", "w", encoding="utf-8") as f:
    for p in payloads:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")

print("saved:", len(payloads), "rows -> llm_input.jsonl")